# Workaround for GeoJSON type bug

Workaround for pydeck 0.1.dev5's [issue processing GeoJSON](https://github.com/uber/deck.gl/issues/3836). Thank you [pancho111203](https://github.com/pancho111203) for filing the bug.

In [ ]:
import pandas as pd

# Load in the JSON data
DATA_URL = 'https://raw.githubusercontent.com/uber-common/deck.gl-data/master/examples/geojson/vancouver-blocks.json'
json = pd.read_json(DATA_URL)
polygon_df = pd.DataFrame()

# Parse the geometry out in Pandas and pass it to deck.gl's PolygonLayer
# I'll make the next release such that you don't have to do this
# manual extraction.
polygon_df['coordinates'] = json['features'].apply(lambda row: row['geometry']['coordinates'])
polygon_df['valuePerSqm'] = json['features'].apply(lambda row: row['properties']['valuePerSqm'])
polygon_df['growth'] = json['features'].apply(lambda row: row['properties']['growth'])
polygon_df.head()

In [ ]:
import pydeck

INITIAL_VIEW_STATE = pydeck.ViewState(
    latitude=49.254,
    longitude=-123.13,
    zoom=11,
    max_zoom=16,
    pitch=45,
    bearing=0
)

geojson = pydeck.Layer(
    'PolygonLayer',
    polygon_df,
    opacity=0.8,
    get_polygon='coordinates',
    stroked=False,
    filled=True,
    extruded=True,
    wireframe=True,
    get_elevation='valuePerSqm / 20',
    get_fill_color='[255, 255, growth * 255]',
    get_line_color=[255, 255, 255],
    pickable=True
)

r = pydeck.Deck(
    layers=[geojson],
    initial_view_state=INITIAL_VIEW_STATE)
r.to_html(iframe_width=1000)